<a href="https://colab.research.google.com/github/FaisalRashid9775/AI_Agents/blob/main/%5B10%5D_Advanced_handoffs.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Handoffs Advanced topics : **

In [ ]:
!pip install -Uq openai-agents

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.2/194.2 kB 5.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 144.4/144.4 kB 9.7 MB/s eta 0:00:00


In [ ]:
from agents import Agent,Runner,OpenAIChatCompletionsModel,AsyncOpenAI,set_tracing_disabled, function_tool, RunContextWrapper
import os
from google.colab import userdata
api_key= userdata.get('GOOGLE_API_KEY')
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")
client : AsyncOpenAI = AsyncOpenAI(
    api_key=api_key,
    base_url="https://generativelanguage.googleapis.com/v1beta/openai/")

lite_model : OpenAIChatCompletionsModel = OpenAIChatCompletionsModel(
    model = 'gemini-1.5-flash',
    openai_client=client
)
pro_model : OpenAIChatCompletionsModel = OpenAIChatCompletionsModel(
    model = 'gemini-2.5-flash',
    openai_client=client
)
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")

import nest_asyncio
nest_asyncio.apply()

In [ ]:
from agents import handoff
physics_agent = Agent(
    name='physics-agent',
    instructions='''You are helpful agent
     Your main responsibility to answer query Physics related only.
    ''',
    model=pro_model
    )


main_agent = Agent(
    name='main-agent',
    instructions='''You are helpful agent.You are only allowed to Answer the question related to physics

      ''',
    model=pro_model,
    handoffs = [handoff(physics_agent,tool_name_override='phy-agent')],
    )


reponse  = Runner.run_sync(main_agent,' what is velocity')

print(reponse.final_output)
print(f'Replaing Agent : {reponse.last_agent.name}')




In physics, **velocity** is a vector quantity that describes the rate at which an object changes its position. It tells you both:

1.  **Speed (magnitude):** How fast the object is moving.
2.  **Direction:** The direction in which the object is moving.

Here's a breakdown:

*   **Vector Quantity:** This is crucial. Unlike speed (which is a scalar and only tells you "how fast"), velocity includes direction. For example, "50 km/h" is a speed, but "50 km/h North" is a velocity.
*   **Rate of Change of Displacement:** It's formally defined as the rate of change of **displacement** over time.
    *   **Displacement** is the shortest distance from the initial to the final position, including direction.
    *   **Distance** is the total path traveled.
*   **Formula:**
    *   **Average Velocity ($\bar{v}$):** $\bar{v} = \frac{\Delta x}{\Delta t}$
        *   $\Delta x$ = change in displacement
        *   $\Delta t$ = change in time
    *   **Instantaneous Velocity ($v$):** The velocity at a 

We can print each time when an agent handoffs to another

In [ ]:
physics_agent = Agent(
    name='physics-agent',
    instructions='''You are helpful agent. Your main responsibility to answer query Physics related only.
    ''',
    model=pro_model
    )
def tranfer_to_physics_agent(context :RunContextWrapper):
  print(f'handoff to physics from main agent')

sub_agent = Agent(
    name='sub_agent',
    instructions='''You are helpful agent. Handoffs to physics for physics related question ''',
    model=lite_model,
    handoffs=[handoff(physics_agent, on_handoff=tranfer_to_physics_agent)]
)
reponse  = Runner.run_sync(sub_agent,' what is velocity')

print(reponse.final_output)
print(f'Replaing Agent : {reponse.last_agent.name}')

handoff to physics from main agent


Velocity is a fundamental concept in physics that describes an object's motion. Here's a breakdown:

1.  **Definition:** Velocity is the rate at which an object changes its position with respect to a reference point, and it includes both the **speed** and the **direction** of the motion.

2.  **Vector Quantity:** This is a crucial distinction. Velocity is a **vector quantity**, meaning it has:
    *   **Magnitude:** How fast the object is moving (its speed).
    *   **Direction:** The path along which the object is moving (e.g., north, south, up, down, or a specific angle).
    *   In contrast, **speed** is a scalar quantity, only having magnitude.

3.  **Formula:**
    *   **Average Velocity ($\vec{v}_{avg}$):** The total **displacement** ($\Delta \vec{x}$) divided by the total time taken ($\Delta t$).
        $\vec{v}_{avg} = \frac{\Delta \vec{x}}{\Delta t}$
        Displacement is the straight-line distance and direction from the initial to the final position.
    *   **Instantaneou

You can print the reason why sub_agent handoffs to other agent.

In [ ]:
# from pydantic import BaseModel
# class Handoff_CLass(BaseModel):
#   topic : str
#   reason:str
from dataclasses import dataclass
@dataclass
class Handoff_CLass:
  reason:str

physics_agent = Agent(
    name='physics-agent',
    instructions='''You are helpful agent. Your main responsibility to answer query Physics related only.
    ''',
    model=pro_model
    )
def handoff_to(context :RunContextWrapper,input:Handoff_CLass):
  print(f'handoff to physics from main agent {input.reason}')

sub_agent = Agent(
    name='sub_agent',
    instructions='''You are helpful agent. Handoffs to physics for physics related question ''',
    model=lite_model,
    handoffs=[handoff(physics_agent,on_handoff=handoff_to, input_type=Handoff_CLass)]
)
reponse  = Runner.run_sync(sub_agent,' what is centripetal force')

print(reponse.final_output)
print(f'Replaing Agent : {reponse.last_agent.name}')



handoff to physics from main agent The user wants to know about centripetal force.


Ah, a fundamental concept in physics! Let's break down **centripetal force**.

In simple terms, **centripetal force is the net force that causes an object to move in a curved or circular path.**

Here's a more detailed explanation:

1.  **Direction:** The key characteristic of centripetal force is that it always points **towards the center of the circular path**. The word "centripetal" itself comes from Latin, meaning "center-seeking."

2.  **What it Does:** An object moving in a circle is continuously changing its *direction* of motion, even if its *speed* is constant. A change in direction is an acceleration. According to Newton's Second Law ($\text{F} = \text{ma}$), for there to be an acceleration, there must be a net force acting on the object. This net force, which causes the object to accelerate towards the center of the circle, is the centripetal force.

3.  **Not a New Type of Force:** It's important to understand that centripetal force isn't a new fundamental force like gravit

# **Input_filter**

control the data which we usually send to our handoff agents.
If we want to do not send tool data or raw data(which is not useful use remove_all_data)

In [ ]:
from agents.extensions import handoff_filters
handoffs=[handoff(physics_agent, input_filter= handoff_filters.remove_all_tools)]


If we want to send selective Data to handoffs agent use.

In [ ]:
from agents import HandoffInputData,handoff
def summarizing_filter(input:HandoffInputData) -> HandoffInputData:
    summarize = 'Physics concept'
    return HandoffInputData(
        input_history=input.input_history + summarize,
        pre_handoff_items=(),
        new_items=()
    )


physics_agent = Agent(name='physics_agent',instructions='''You are helpful agent. ''',model=lite_model)


sub_agent = Agent(name='sub_agent',instructions='''You are helpful agent. Use the physics_agent to answer
 questions about physics. ''',
                  model=lite_model,
                  handoffs=[handoff(physics_agent, input_filter=summarizing_filter, is_enabled = True)])   # sumarzing filter is a function


# **Structure Output** :

Through this you can tell LLM in which structure output have to gives.

In [ ]:
from agents import Agent,Runner, AsyncOpenAI, OpenAIChatCompletionsModel, set_tracing_disabled, function_tool
from google.colab import userdata
from pydantic import BaseModel,Field

class Users(BaseModel):
    name: str |None = Field(default=None,description='Name of the person')
    age: int |None = Field(default=None,description='Age of the person')
    interest: str |None = Field(default=None,description='Interest of the person')
    education:str |None = Field(default=None,description='Education of the person')
    graduation_year:int |None = Field(default=1998,description='Graduation year of the person')
    skills:list[str] |None = Field(default=None,description='Skills of the person')
gemini_api_key = userdata.get('GOOGLE_API_KEY')
import nest_asyncio
nest_asyncio.apply()

client = AsyncOpenAI(api_key=gemini_api_key,
                     base_url="https://generativelanguage.googleapis.com/v1beta/openai/")

lite_model  = OpenAIChatCompletionsModel(
    model='gemini-2.5-flash',
    openai_client=client
)
set_tracing_disabled(disabled=True)

agent = Agent(name='agnet',instructions='Extract person information from the users message',model=lite_model,output_type=Users)
response  = Runner.run_sync(agent,'I am Ali 25 years old and i am interested in coding and i did my master in CS in 2024 and my skills')
print(response.final_output)


name='Ali' age=25 interest='coding' education='Master in CS' graduation_year=2024 skills=[]
